# Objective
The purpose is to create a demonstable prototype that mines purchase data and predicts categories similar to the input.
For elucidations sake we will divide the summary problem into sub problems.
*  Problem 1 -Given item A predict item B which is most associated through purchase patterns.
*  Problem 2 -Given item A predict a category which has the most similarity to the item's category


# Packages required
* Pandas for data frame
* Numpy for arrays
* Scipy for sparse jaccardian

In [1]:
import pandas as pd
import numpy as np
from scipy import sparse

# Problem 1

# Import Data

In [2]:
#data = pd.read_csv('../data/FMCGSales.csv', names = ['BillId','ItemId','ItemName','Level1','Level2','Level3','Level4','Level5','Level6'] )


In [3]:
data = pd.read_csv('../data/1LakhFMCGSalesWithCategory.csv', names = ['BillId','ItemId','ItemName','Level1','Level2','Level3','Level4','Level5','Level6'] )


In [4]:
#Dummy for pivot table
data['dummy'] = 1

In [5]:
data.head()

,BillId,ItemId,ItemName,Level1,Level2,Level3,Level4,Level5,Level6,dummy
0,121210,52344,GULABARI ROSE GLOW CLEANSER 100ML,FMCG,FMCG NON FOOD,PERSONAL CARE,SKIN CARE,FACE WASH,CLEANSING,1
1,121270,59909,J&J BABY WIPES 80PCS,FMCG,FMCG NON FOOD,PERSONAL CARE,BABY CARE,HYGIENE,WIPES,1
2,121321,103829,SAT ISABGOL 100GM,FMCG,AYUSH,AYURVEDIC,POWDER,CHURAN,NaN,1
3,121360,30225,COLGATE GEL MAXFRESH RED 150GM,FMCG,FMCG NON FOOD,PERSONAL CARE,ORAL CARE,TOOTHPASTE,GEL,1
4,121788,91629,PATANJALI DANT KANTI MEDI ORAL GEL 100GM,FMCG,FMCG NON FOOD,PERSONAL CARE,ORAL CARE,TOOTHPASTE,GEL,1


# Data Exploration

In [6]:
data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
BillId,100000.0,1.111762e+07,7.275864e+06,129.0,1533840.0,16334841.5,16439740.75,16544632.0
ItemId,100000.0,6.512184e+04,3.326999e+04,9254.0,33533.0,59909.0,91113.00,127464.0
dummy,100000.0,1.000000e+00,0.000000e+00,1.0,1.0,1.0,1.00,1.0


In [7]:
#is any row NULL ?
data.isnull().any().any(), data.shape

(True, (100000, 10))

In [8]:
#describe nullness
data.isnull().sum(axis=0)

BillId          0
ItemId          0
ItemName        0
Level1          0
Level2          0
Level3          0
Level4          0
Level5         12
Level6      10241
dummy           0
dtype: int64

In [9]:
data = data.dropna()

In [10]:
#is any row NULL ?
data.isnull().any().any(), data.shape

(False, (89759, 10))

Level 5 has 5 null values
Level 6 has 1286 null Values
Use data.dropna() to drop null rows if we are using that. Dont know if filling null values will solve the problem because the null values can have many values. Perhaps clustering can be employed to label these products first.

In [54]:
#test for loc arbitrary item Id 220862
data.loc[data['BillId'] == 1238]

,BillId,ItemId,ItemName,Level1,Level2,Level3,Level4,Level5,Level6,dummy
56,1238,101287,ROGAN BADAM SIRIN 25ML,FMCG,FMCG FOOD,GROCERY,COOKING MEDIUM,OIL,ALMOND OIL,1
119,1238,52331,GULABARI 250ML,FMCG,FMCG NON FOOD,PERSONAL CARE,SKIN CARE,LOTION,BEAUTY & NOURISHMENT,1
200,1238,51605,GOOD KNIGHT ADVANCE REFILL 45ML,FMCG,FMCG NON FOOD,HOME CARE,PEST CONTROL,LIQUID,MOSQUITO REPELLENT,1
234,1238,54581,HIMALAYA FACE WASH PURIFYING NEEM 50ML,FMCG,FMCG NON FOOD,PERSONAL CARE,SKIN CARE,FACE WASH,NEEM,1
313,1238,89822,PAMPERS BD S 22PCS,FMCG,FMCG NON FOOD,PERSONAL CARE,BABY CARE,HYGIENE,DIAPERS S,1


In [61]:
data.loc[data['Level5'] == 'Soup\\\']

SyntaxError: invalid syntax (<ipython-input-61-72a23a1501ab>, line 1)

In [66]:
data[data['ItemName'].str.contains("MAGGI")].sort_values(by='ItemName')

,BillId,ItemId,ItemName,Level1,Level2,Level3,Level4,Level5,Level6,dummy
67630,16415038,71542,MAGGI NOODLES 100GM,FMCG,FMCG FOOD,PROCESSED FOOD,NOODLES & PASTA,NOODLES,OTHERS,1
25232,1539832,71543,MAGGI NOODLES 140GM,FMCG,FMCG FOOD,PROCESSED FOOD,NOODLES & PASTA,NOODLES,OTHERS,1
69502,16422347,71553,MAGGI NOODLES 70GM,FMCG,FMCG FOOD,PROCESSED FOOD,NOODLES & PASTA,NOODLES,OTHERS,1
78606,16454859,71553,MAGGI NOODLES 70GM,FMCG,FMCG FOOD,PROCESSED FOOD,NOODLES & PASTA,NOODLES,OTHERS,1
78436,16457893,71553,MAGGI NOODLES 70GM,FMCG,FMCG FOOD,PROCESSED FOOD,NOODLES & PASTA,NOODLES,OTHERS,1
78434,16457794,71553,MAGGI NOODLES 70GM,FMCG,FMCG FOOD,PROCESSED FOOD,NOODLES & PASTA,NOODLES,OTHERS,1
77908,16457149,71553,MAGGI NOODLES 70GM,FMCG,FMCG FOOD,PROCESSED FOOD,NOODLES & PASTA,NOODLES,OTHERS,1
77896,16456602,71553,MAGGI NOODLES 70GM,FMCG,FMCG FOOD,PROCESSED FOOD,NOODLES & PASTA,NOODLES,OTHERS,1
77894,16456432,71553,MAGGI NOODLES 70GM,FMCG,FMCG FOOD,PROCESSED FOOD,NOODLES & PASTA,NOODLES,OTHERS,1
77513,16454033,71553,MAGGI NOODLES 70GM,FMCG,FMCG FOOD,PROCESSED FOOD,NOODLES & PASTA,NOODLES,OTHERS,1


In [42]:
listed = data['Level5'].unique()  


In [43]:
listed_df = pd.DataFrame(listed)

In [44]:
listed_df.sort_values(by = 0).to_csv('../data/listof_level5_values.csv')

# Pivot Table

In [15]:
matrix = data.pivot_table(values='dummy',index ='BillId', columns ='Level5')
matrix.head()

Level5,ACCESSORIES,AEROSOL,BALM,BAR,BATH,BODY WASH,BOURBON,BREAKFAST SPREAD,BUTTER,CAKE,...,TAPE,TOILET TISSUE,TOOTH BRUSH,TOOTH POWDER,TOOTHPASTE,WAFERS,WATER,WAX,WHOLE SPICE,WIPES
BillId,,,,,,,,,,,,,,,,,,,,,
129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1214,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
#matrix_dummy[['TOOTH BRUSH','TOOTH PASTE']].to_csv('../data/toothlistforcheck.csv')

In [17]:
matrix.shape

(65437, 105)

In [49]:
matrix.to_csv('../data/matrixpivottable.csv')

In [18]:
matrix_dummy = matrix.copy().fillna(0)

In [19]:
matrix_dummy.head()

Level5,ACCESSORIES,AEROSOL,BALM,BAR,BATH,BODY WASH,BOURBON,BREAKFAST SPREAD,BUTTER,CAKE,...,TAPE,TOILET TISSUE,TOOTH BRUSH,TOOTH POWDER,TOOTHPASTE,WAFERS,WATER,WAX,WHOLE SPICE,WIPES
BillId,,,,,,,,,,,,,,,,,,,,,
129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
177,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
179,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1214,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1216,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Skewed Jaccardian
We define a jaccardian as intersection over union. A skewed Jaccardian is intersection over set A

### method from http://na-o-ys.github.io/others/2015-11-07-sparse-vector-similarities.html

In [20]:
#needs parameter in scipy.sparse.csc_matrix
type(matrix_dummy)

pandas.core.frame.DataFrame

# Experiment

In [21]:
sparse_matrix = sparse.csc_matrix(matrix_dummy)

In [22]:
type(sparse_matrix)

scipy.sparse.csc.csc_matrix

In [23]:
def jaccard_similarities(mat):
    cols_sum = mat.getnnz(axis=0)
    ab = mat.T * mat

    # for rows
    aa = np.repeat(cols_sum, ab.getnnz(axis=0))
    # for columns
    bb = cols_sum[ab.indices]

    similarities = ab.copy()
    similarities.data /= (aa + bb - ab.data)

    return similarities

In [24]:
jaccard_similarities =  jaccard_similarities(sparse_matrix)

In [25]:
jaccard_similarities

<105x105 sparse matrix of type '<class 'numpy.float64'>'
	with 4267 stored elements in Compressed Sparse Row format>

In [26]:
print(jaccard_similarities)

  (0, 96)	0.0006435006435006435
  (0, 65)	0.0006485084306095979
  (0, 13)	0.0005790387955993051
  (0, 39)	0.0006345177664974619
  (0, 90)	0.0006472491909385113
  (0, 38)	0.0006009615384615385
  (0, 67)	0.0005691519635742744
  (0, 92)	0.001722158438576349
  (0, 33)	0.0006273525721455458
  (0, 36)	0.0006480881399870382
  (0, 27)	0.0006480881399870382
  (0, 11)	0.0012782275244993609
  (0, 53)	0.0010758472296933835
  (0, 64)	0.0021968365553602814
  (0, 48)	0.0023752969121140144
  (0, 6)	0.0023594180102241447
  (0, 7)	0.0022114108801415304
  (0, 59)	0.0038293216630196935
  (0, 71)	0.0033495736906211937
  (0, 15)	0.002352018816150529
  (0, 8)	0.0009256402344955261
  (0, 26)	0.007404273323460969
  (0, 17)	0.0012634238787113076
  (0, 29)	0.0055936943808797355
  (0, 81)	0.00129366106080207
  :	:
  (101, 12)	0.0004945598417408506
  (101, 8)	0.005802707930367505
  (101, 16)	0.009719522354901417
  (101, 11)	0.010301692420897719
  (101, 69)	0.0020348837209302325
  (101, 59)	0.0004484304932735426
  

In [41]:
matrix.columns.to_csv('../data/listoflevel5.csv')

AttributeError: 'Index' object has no attribute 'to_csv'

In [28]:
jaccardian = pd.DataFrame(jaccard_similarities.toarray(), index = matrix.columns,columns = matrix.columns)

In [29]:
jaccardian.head()

Level5,ACCESSORIES,AEROSOL,BALM,BAR,BATH,BODY WASH,BOURBON,BREAKFAST SPREAD,BUTTER,CAKE,...,TAPE,TOILET TISSUE,TOOTH BRUSH,TOOTH POWDER,TOOTHPASTE,WAFERS,WATER,WAX,WHOLE SPICE,WIPES
Level5,,,,,,,,,,,,,,,,,,,,,
ACCESSORIES,1.000000,0.0,0.004411,0.001195,0.0,0.004416,0.002359,0.002211,0.000926,0.006952,...,0.000615,0.000644,0.013193,0.000000,0.010522,0.0,0.001449,0.0,0.0,0.0
AEROSOL,0.000000,1.0,0.000000,0.000000,0.0,0.000000,0.001675,0.001096,0.000000,0.000415,...,0.000000,0.000000,0.000952,0.000000,0.000614,0.0,0.000000,0.0,0.0,0.0
BALM,0.004411,0.0,1.000000,0.000635,0.0,0.001342,0.000408,0.004172,0.001274,0.003559,...,0.001312,0.000000,0.004189,0.000000,0.008497,0.0,0.000895,0.0,0.0,0.0
BAR,0.001195,0.0,0.000635,1.000000,0.0,0.000000,0.002584,0.000000,0.001077,0.000841,...,0.000000,0.000000,0.005828,0.006536,0.008128,0.0,0.000000,0.0,0.0,0.0
BATH,0.000000,0.0,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0


In [30]:
jaccardian.shape

(105, 105)

In [31]:
type(jaccardian)

pandas.core.frame.DataFrame

In [32]:
df = pd.DataFrame(np.triu(jaccardian, 1), columns=jaccardian.columns, index=jaccardian.index)

In [33]:
type(df), df.shape

(pandas.core.frame.DataFrame, (105, 105))

In [34]:
filtered = jaccardian[jaccardian[jaccardian < 1] > 0.1].stack()

In [35]:
filtered

Level5       Level5     
TOOTH BRUSH  TOOTHPASTE     0.163645
TOOTHPASTE   TOOTH BRUSH    0.163645
dtype: float64

In [74]:
jaccardian.loc[['TOOTH POWDER'],['TOOTH BRUSH']]

Level5,TOOTH BRUSH
Level5,
TOOTH POWDER,0.001553


In [72]:
jaccardian.loc['OATS'].sort_values(ascending=False)

Level5
OATS                      1.000000
OIL                       0.000294
COOKIES                   0.000000
CORNFLAKES                0.000000
COTTON                    0.000000
CRACKER                   0.000000
CREAM                     0.000000
CREAM & OINTMENTS         0.000000
DAIRY                     0.000000
DEODORIZERS               0.000000
DETERGENT POWDER          0.000000
DIGESTIVE CARE            0.000000
DISINFECTANT              0.000000
WIPES                     0.000000
FACE GEL                  0.000000
FACE PACK                 0.000000
FACE SCRUB                0.000000
FACE WASH                 0.000000
FACIAL TISSUES            0.000000
FIRST Aid                 0.000000
FRAGRANCES                0.000000
FUNCTIONAL BEVERAGE       0.000000
GEL                       0.000000
GIFT PACK                 0.000000
GLUCOSE                   0.000000
EYE COSMETICS & BEAUTY    0.000000
CONTRACEPTIVE             0.000000
HAND                      0.000000
CONDITIONER  